In [ ]:
!pip install patool --quiet

import os, warnings

warnings.filterwarnings("ignore")

In [2]:
import tensorflow as tf

import os, patoolib, urllib

In [ ]:
urli = "https://github.com/dicodingacademy/assets/releases/download/release/"

dataset = "rockpaperscissors.zip"

urllib.request.urlretrieve(os.path.join(urli, dataset), dataset)

In [ ]:
content = "/content/rockpaperscissors.zip"

patoolib.extract_archive(content)

In [5]:
rooter = "/content/rockpaperscissors/rockpaperscissors/rps-cv-images"

os.listdir(rooter)

['README_rpc-cv-images.txt', 'scissors', 'rock', 'paper']

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

rescale = 1.0 / 255.0; input_image = (150, 150)

class_label = "categorical"; batches = 18

validation_split = 0.4

creator = ImageDataGenerator(
  rescale = rescale,
  rotation_range = 20,
  zoom_range = 0.2,
  shear_range = 0.2,
  fill_mode = "nearest",
  validation_split = validation_split
)

train = creator.flow_from_directory(
  rooter,
  target_size = input_image,
  batch_size = batches,
  class_mode = class_label,
  subset = "training"
)

In [7]:
train.class_indices

{'paper': 0, 'rock': 1, 'scissors': 2}

In [8]:
label = list(train.class_indices.keys())

label

['paper', 'rock', 'scissors']

In [9]:
train.num_batches

73

In [ ]:
validation = creator.flow_from_directory(
  rooter,
  target_size = input_image,
  batch_size = batches,
  class_mode = class_label,
  subset = "validation"
)

In [11]:
from tensorflow.keras.models import Sequential

input_shape = (150, 150, 3)

from tensorflow.keras.layers import Conv2D, MaxPool2D

from tensorflow.keras.layers import Dropout, Flatten, Dense

model = Sequential([
  Conv2D(16, 3, activation="relu", padding="same", input_shape=input_shape),
  MaxPool2D((2, 2)),
  Conv2D(32, 3, activation="relu", padding="same"),
  MaxPool2D((2, 2)),
  Conv2D(64, 3, activation="relu", padding="same"),
  MaxPool2D((2, 2)),
  Conv2D(128, 3, activation="relu", padding="same"),
  MaxPool2D((2, 2)),
  Dropout(0.3),
  Flatten(),
  Dense(128, activation="relu"),
  Dense(3, activation="softmax")
])

In [12]:
loss = "categorical_crossentropy"

model.compile(optimizer="adam", loss=loss, metrics=["accuracy"])

In [13]:
from tensorflow.keras.callbacks import Callback, EarlyStopping

class Matrit(Callback):
  def on_epoch_end(self, epoch, logs=None):
    if logs.get("accuracy") >= 0.95 and logs.get("val_accuracy") >= 0.95:
      self.model.stop_training = True

callbacks = Matrit()

stopper = EarlyStopping(monitor="val_loss", patience=3)

In [ ]:
epochs = 19

monitor = [callbacks, stopper]

plot = model.fit(train, validation_data=validation, epochs=epochs, callbacks=monitor)

In [15]:
model.evaluate(train, verbose=0)

[0.07975878566503525, 0.9687975645065308]

In [16]:
model.evaluate(validation, verbose=0)

[0.11562804132699966, 0.9610983729362488]

In [17]:
import tensorflow.keras.utils as utils

import os

tester = "/content/rockpaperscissors/rockpaperscissors/rock/00nKV8oHuTGi20gq.png"

i = utils.load_img(tester, target_size=(150, 150))

i = utils.img_to_array(i)

i = tf.expand_dims(i, axis=0)

i = tf.cast(i / 255.0, tf.float32)

result = model.predict(i, verbose=0)[0]

result = label[result.argmax()]

result

'rock'